In [1]:
import numpy as np
from PIL import Image
import cv2
import matplotlib.pyplot as plt

In [2]:
import os
TRAIN_DIR = "MiddEval3/trainingQ/"
IMAGE_LEFT, IMAGE_RIGHT = "im0.png", "im1.png"
TRAIN_LISTS = [filename for filename in os.listdir(TRAIN_DIR)]
print(TRAIN_LISTS)

['ArtL', 'Motorcycle', 'MotorcycleE', 'Vintage', 'Playroom', 'Jadeplant', 'Shelves', 'Piano', 'PianoL', 'Adirondack', 'Teddy', 'Recycle', 'PlaytableP', 'Pipes', 'Playtable']


In [3]:
def distortion(img1, img2):
    sift = cv2.xfeatures2d.SIFT_create()
    # find the keypoints and descriptors with SIFT
    kp1, des1 = sift.detectAndCompute(img1,None)
    kp2, des2 = sift.detectAndCompute(img2,None)
    # FLANN parameters
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    
    good = []
    pts1 = []
    pts2 = []
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.85*n.distance:
            good.append(m)
            pts2.append(kp2[m.trainIdx].pt)
            pts1.append(kp1[m.queryIdx].pt)
    
    pts1 = np.int32(pts1)
    pts2 = np.int32(pts2)
    F, mask = cv2.findFundamentalMat(pts1,pts2,cv2.RANSAC)
    # We select only inlier points
    pts1 = pts1[mask.ravel()==1]
    pts2 = pts2[mask.ravel()==1]
    pts1 = pts1.reshape(-1, 1, 2)
    pts2 = pts2.reshape(-1, 1, 2)
    
    return pts1, pts2, F, mask

# distortion()

In [4]:
def rectify_img(imgL, imgR, output_predix):
    img1 = cv2.imread(imgL,1)  #queryimage # left image
    img2 = cv2.imread(imgR,1) #trainimage # right image
    imgL_pts, imgR_pts, F_mat, mask = distortion(img1, img2)
    imsize = (img1.shape[1], img1.shape[0])
    print(imsize)
    _, H1, H2 = cv2.stereoRectifyUncalibrated(imgL_pts,
                                              imgR_pts,
                                              F_mat, imsize)
    dst11 = cv2.warpPerspective(img1,H1,imsize)
    dst22 = cv2.warpPerspective(img2,H2,imsize)
    print(dst11.shape, dst22.shape)
    out_l, out_r = output_predix+"L.png", output_predix+"R.png"
    cv2.imwrite(out_l, dst11)
    cv2.imwrite(out_r, dst22)
    return [out_l, out_r]

In [5]:
def stereo_match(imgL, imgR, kernel=6, search_range=30, name="default.png"):
    w, h = imgL.size
    left, right = np.asarray(imgL), np.asarray(imgR)
    depth = np.zeros((h, w), np.uint8) # PIL and numpy have different indexing systems.
    
    #depth.shape = h, w
    half_kernel = kernel//2
    depth_offset = 255//search_range
    for y in range(half_kernel, h-half_kernel):
        print(".", end="", flush=True)
        for x in range(half_kernel, w-half_kernel):
            optimal_offset = 0
            ssd_min = 256*256
            for search_idx in range(search_range):
                ssd = 0
                for u in range(-half_kernel, half_kernel):
                    for v in range(-half_kernel, half_kernel):
                        ssd += (int(left[y+u, x+v]) - int(right[y+u, x+v-search_idx]))**2
                if ssd < ssd_min:
                    ssd_min = ssd
                    optimal_offset = search_idx
            depth[y, x] = optimal_offset * depth_offset
    print(depth)
    Image.fromarray(depth).save(name)

In [6]:
Rectified_list = []
for train in TRAIN_LISTS:
    train_dir = TRAIN_DIR + train + "/"
    print(train_dir)
    left_img = train_dir + IMAGE_LEFT
    right_img = train_dir + IMAGE_RIGHT
    iml = Image.open(left_img).convert("L")
    imr = Image.open(right_img).convert("L")
    # res = rectify_img(left_img, right_img, train_dir+"Rectified")
    # Rectified_list.append(res)
    # stereo_match(imgL=iml, imgR=imr, name=train_dir+"output.png")
# print(Rectified_list)

MiddEval3/trainingQ/ArtL/


NameError: name 'res' is not defined

In [8]:
iml = Image.open('MiddEval3/trainingQ/MotorcycleE/RectifiedL.png').convert("L")
imr = Image.open('MiddEval3/trainingQ/MotorcycleE/RectifiedR.png').convert("L")
stereo_match(imgL=iml, imgR=imr, name="MiddEval3/trainingQ/MotorcycleE/output_v2.png")

...........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]
